In [ ]:
# import libraries
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pqdm.processes import pqdm
from tqdm.notebook import tqdm

import utils

In [ ]:
# load wind farm
x, y, wfm_high, wfm_low, wfm_lossless = utils.get_wind_farm_model()
nt = len(x)
power_installed = wfm_high.windTurbines.powerCtFunction.power_ct_tab[0].max() / 1e9 * nt

In [ ]:
# visualise wind distribution
_ = wfm_low.site.plot_wd_distribution(n_wd=8, ws_bins=[0, 5, 10, 15, 20, 25, 30])

In [ ]:
# define simulation function
def run_sim(yaw=0, ws=np.arange(0, 31, 5)):
    sim_res = wfm_low(
        x=x,
        y=y,
        tilt=0,
        yaw=yaw,
        n_cpu=None,
        ws=ws,
        wd=270,
    )
    return sim_res

In [ ]:
# run simulation
sim_res = run_sim(yaw=0)
print(
    f"LCoE [USD/MWh]: {(utils.OPEX_GWy + utils.CAPEX_GW / utils.LIFESPAN) * power_installed / (sim_res.aep().sum().values * 1000):,.3f}"
)
print(
    f"Capacity factor [%]: {sim_res.aep().sum().values / (power_installed * 365.25 * 24)*100:,.2f}"
)
sim_res

In [ ]:
# initialise dataset
yaw_ind = {f"yaw_{i}": [-5, 0, 5] for i in sim_res.wt.values}
ds = xr.Dataset(
    data_vars={
        "lcoe": ([f"yaw_{i}" for i in sim_res.wt.values], np.zeros((3,) * nt)),
        "cap_fac": ([f"yaw_{i}" for i in sim_res.wt.values], np.zeros((3,) * nt)),
        "aep": (
            ["wt"] + [f"yaw_{i}" for i in sim_res.wt.values],
            np.zeros((nt,) + (3,) * nt),
        ),
    },
    coords={"wd": list(sim_res.wt.values)} | yaw_ind,
)
ds

In [ ]:
# assess across range of yaw combinations

# define yaw options for consideration
yaw_options = list(product(*yaw_ind.values()))

# loop over all yaw option values
for yaw_option in tqdm(yaw_options):
    # run simulation
    sim_res = run_sim(yaw=yaw_option)

    # calculate values and store in dataframe
    ds["lcoe"].loc[*yaw_option] = (
        (utils.OPEX_GWy + utils.CAPEX_GW / utils.LIFESPAN)
        * power_installed
        / (sim_res.aep().sum().values * 1000)
    )
    ds["cap_fac"].loc[*yaw_option] = sim_res.aep().sum().values / (
        power_installed * 365.25 * 24
    )
    ds["aep"].loc[:, *yaw_option] = sim_res.aep().sum(["wd", "ws"]).values

In [ ]:
# flatten and sort values
flat_lcoe = ds.lcoe.stack(all_dims=yaw_ind.keys())
yaw_sorted = flat_lcoe["all_dims"][np.argsort(flat_lcoe.values)]
display(yaw_sorted)
yaw_opt = yaw_sorted[0].values.tolist()

In [ ]:
# plot distribution of lcoe values
_ = ds.lcoe.plot.hist(bins=25)

In [ ]:
# plot distribution of capacity factor values
_ = ds.cap_fac.plot.hist(bins=25)

In [ ]:
# run simulation using optimum yaw
sim_res_opt = run_sim(yaw=yaw_opt)
print(
    f"LCoE [USD/MWh]: {(utils.OPEX_GWy + utils.CAPEX_GW / utils.LIFESPAN) * power_installed / (sim_res_opt.aep().sum().values * 1000):,.3f}"
)
print(
    f"Capacity factor [%]: {sim_res_opt.aep().sum().values / (power_installed * 365.25 * 24)*100:,.2f}"
)

In [ ]:
# plot flow map
fig, ax = plt.subplots()
fm = run_sim(yaw=yaw_opt, ws=12).flow_map()
fm.plot_wake_map(ax=ax)
ax.grid()
fig.tight_layout()
fm